In [1]:
# Install required packages
!pip install -q streamlit
!pip install -q torch
!pip install -q transformers
!pip install -q pyngrok


    sys-platform (=="darwin") ; extra == 'objc'
                 ~^
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
huggingface-hub 0.27.0 requires fsspec>=2023.5.0, which is not installed.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
    sys-platform (=="darwin") ; extra == 'objc'
                 ~^
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
easyocr 1.7.2 requires torchvision>=0.5, which is not installed.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
    sys-platform (=="darwin") ; extra == 'objc'
                 ~^

[notice] A new release of pip is available: 24.3.1 -> 25.0

In [9]:
! pip install git+https://github.com/huggingface/diffusers

  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-nw_5_iqe
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-nw_5_iqe
  Resolved https://github.com/huggingface/diffusers to commit aad69ac2f323734a083d66fa89197bf7d88e5a57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [10]:
import sys

!{sys.executable} -m pip install huggingface_hub
! pip install bitsandbytes
!pip install xformers

In [11]:
from huggingface_hub import login
login(token="")

In [14]:
%%writefile app.py
import streamlit as st
import torch
from diffusers import StableDiffusionPipeline
import io
import platform
import logging
import os
from pathlib import Path

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Page config
st.set_page_config(
    page_title="Text to Image Generator",
    page_icon="🎨",
    layout="wide"
)

# Custom CSS to improve the interface
st.markdown("""
    <style>
        .stTextInput > label {
            font-size: 20px;
            font-weight: bold;
        }
        .stButton > button {
            width: 100%;
            height: 50px;
            font-size: 18px;
        }
        .suggestion-button {
            margin: 5px;
            padding: 10px;
        }
    </style>
""", unsafe_allow_html=True)

# Define constants
MODEL_ID = "OFA-Sys/small-stable-diffusion-v0/"
MODEL_DIR = Path.home() / ".stable_diffusion_models"

# Define suggested prompts
SUGGESTED_PROMPTS = {
    "Fantasy": [
        "A magical crystal castle floating in the clouds with rainbow bridges",
        "A wise ancient dragon reading books in a mystical library",
        "A enchanted forest with glowing mushrooms and fairy lights"
    ],
    "Landscapes": [
        "A serene mountain lake at sunset with snow-capped peaks",
        "Rolling hills of lavender fields under a starry night sky",
        "A tropical beach paradise with crystal clear waters and palm trees"
    ],
    "Abstract": [
        "A surreal fusion of geometric shapes and flowing liquid colors",
        "An abstract representation of music using swirling colors and light",
        "A dream-like composition of floating objects and nebulous forms"
    ]
}

def check_system_compatibility():
    """Check system compatibility and requirements"""
    system_info = {
        "Python Version": platform.python_version(),
        "PyTorch Version": torch.__version__,
        "CUDA Available": torch.cuda.is_available(),
        "System Platform": platform.system(),
        "Model Directory": str(MODEL_DIR),
    }

    if torch.cuda.is_available():
        system_info["CUDA Version"] = torch.version.cuda
        system_info["GPU Device"] = torch.cuda.get_device_name(0)

    return system_info

def is_model_downloaded():
    """Check if the model is already downloaded"""
    return (MODEL_DIR / "model_index.json").exists()

def download_model():
    """Download the model to local storage"""
    try:
        logger.info(f"Downloading model to {MODEL_DIR}")
        MODEL_DIR.mkdir(parents=True, exist_ok=True)

        pipe = StableDiffusionPipeline.from_pretrained(
            MODEL_ID,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            safety_checker=None
        )

        pipe.save_pretrained(str(MODEL_DIR))
        logger.info("Model downloaded successfully")
        return True

    except Exception as e:
        logger.error(f"Error downloading model: {str(e)}")
        return False

@st.cache_resource
def load_model():
    """Load the model from local storage or download if necessary"""
    try:
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model_path = str(MODEL_DIR) if is_model_downloaded() else MODEL_ID

        logger.info(f"Loading model from {model_path} on {device}")

        pipe = StableDiffusionPipeline.from_pretrained(
            model_path,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            safety_checker=None
        )

        pipe = pipe.to(device)

        if hasattr(pipe, 'enable_attention_slicing'):
            pipe.enable_attention_slicing()

        logger.info("Model loaded successfully")
        return pipe

    except Exception as e:
        logger.error(f"Error loading model: {str(e)}")
        raise Exception(f"Failed to load the model: {str(e)}")

def generate_image(prompt, num_inference_steps=50, guidance_scale=7.5):
    """Generate image from text prompt"""
    try:
        pipe = load_model()
        device = "cuda" if torch.cuda.is_available() else "cpu"

        with torch.autocast(device):
            image = pipe(
                prompt,
                num_inference_steps=num_inference_steps,
                guidance_scale=guidance_scale,
            ).images[0]

        return image

    except Exception as e:
        logger.error(f"Error generating image: {str(e)}")
        raise Exception(f"Failed to generate image: {str(e)}")

def display_suggested_prompts():
    """Display suggested prompts organized by category"""
    st.subheader("✨ Suggested Prompts")
    st.write("Click on any prompt to use it!")

    tabs = st.tabs(list(SUGGESTED_PROMPTS.keys()))

    for tab, (category, prompts) in zip(tabs, SUGGESTED_PROMPTS.items()):
        with tab:
            for prompt in prompts:
                if st.button(prompt, key=f"btn_{prompt}", use_container_width=True):
                    return prompt
    return None

def main():
    st.title("🎨 Text to Image Generator")

    # Display system information in an expander
    with st.expander("System Information"):
        system_info = check_system_compatibility()
        for key, value in system_info.items():
            st.write(f"**{key}:** {value}")

    # Model download section
    st.subheader("Model Management")
    col1, col2 = st.columns([1, 2])

    with col1:
        if is_model_downloaded():
            st.success("✅ Model is downloaded locally")
        else:
            st.warning("⚠️ Model not downloaded")

        if st.button("Download Model Locally"):
            with st.spinner("📥 Downloading model... This may take several minutes..."):
                if download_model():
                    st.success("✅ Model downloaded successfully!")
                    st.rerun()  # Refresh the page to update the model status
                else:
                    st.error("❌ Failed to download model")

    with col2:
        st.info(
            "Downloading the model locally will improve generation speed and allow offline use. "
            f"The model will be stored in: {MODEL_DIR}"
        )

    st.write("---")

    # Add suggested prompts section
    selected_prompt = display_suggested_prompts()

    st.write("---")
    st.write("Enter your prompt below or use one of the suggested prompts above!")

    # Text input for the prompt
    prompt = st.text_area(
        "Enter your prompt:",
        value=selected_prompt if selected_prompt else "",
        height=100,
        placeholder="A serene landscape with mountains and a lake at sunset..."
    )

    # Advanced options in an expander
    with st.expander("Advanced Options"):
        col1, col2 = st.columns(2)
        with col1:
            num_steps = st.slider(
                "Number of inference steps",
                min_value=20,
                max_value=100,
                value=50,
                help="Higher values = better quality but slower generation"
            )
        with col2:
            guidance = st.slider(
                "Guidance scale",
                min_value=1.0,
                max_value=20.0,
                value=7.5,
                step=0.5,
                help="How closely to follow the prompt. Higher values = more literal interpretation"
            )

    # Generate button
    if st.button("Generate Image") or selected_prompt:
        if not prompt:
            st.error("Please enter a prompt first!")
            return

        try:
            with st.spinner("🎨 Generating your image... This might take a minute..."):
                image = generate_image(prompt, num_steps, guidance)

                # Display the generated image
                st.success("✨ Image generated successfully!")
                st.image(image, caption=prompt, width=400)

                # Add download button
                buf = io.BytesIO()
                image.save(buf, format="PNG")
                st.download_button(
                    label="Download Image",
                    data=buf.getvalue(),
                    file_name="generated_image.png",
                    mime="image/png"
                )

        except Exception as e:
            st.error("⚠️ Generation Error")
            st.error(str(e))
            st.info("Try the following troubleshooting steps:\n"
                   "1. Check your internet connection\n"
                   "2. Verify that all dependencies are correctly installed\n"
                   "3. Try a different prompt\n"
                   "4. Reduce the number of inference steps")

if __name__ == "__main__":
    main()

Overwriting app.py


In [15]:
from pyngrok import ngrok

# Set authentication token if you haven't already done so
ngrok.set_auth_token("2sLUuTzXl5fi3F3gyZJ9TmE4HUx_nLDPWkSpJnjy9LpBVvKM")

# Start Streamlit server on a specific port
!nohup streamlit run app.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://f938-34-82-44-240.ngrok-free.app
